# Load Data

In [37]:
# import modules
import pandas as pd
import numpy as np
import sqlalchemy as sqla
import pickle
from sqlalchemy import create_engine

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn import metrics

import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

import pymysql
from sshtunnel import SSHTunnelForwarder
from datetime import datetime, timedelta

import spacy

# Load pre-trained word embeddings (e.g., spaCy's medium English model)
nlp = spacy.load("en_core_web_md")


from shapely.geometry import MultiPolygon, Polygon
from shapely.ops import nearest_points
import geopandas as gpd
from shapely import wkt

In [39]:
ssh_host = '137.43.49.79'
ssh_port = 22
ssh_username = 'student'
ssh_password = 'Ucd-cs-2023!'

mysql_user = 'root'
mysql_password = 'Group@18'
mysql_db = 'smartcityexplorer'

with SSHTunnelForwarder(
    (ssh_host, ssh_port),
    ssh_username=ssh_username,
    ssh_password=ssh_password,
    remote_bind_address=('127.0.0.1', 3306)
) as tunnel:
    conn = pymysql.connect(
        host='127.0.0.1',
        user=mysql_user,
        password=mysql_password,
        db=mysql_db,
        port=tunnel.local_bind_port
    )
    
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM venue_static")
    results = cursor.fetchall()

    # Convert the results to a pandas DataFrame
    import pandas as pd
    df_venue_static = pd.DataFrame(results, columns=[column[0] for column in cursor.description])
    
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM venue_timings")
    results = cursor.fetchall()
    df_venue_timings = pd.DataFrame(results, columns=[column[0] for column in cursor.description])

    cursor = conn.cursor()
    cursor.execute("SELECT * FROM venue_merged")
    results = cursor.fetchall()
    df_venue_merged = pd.DataFrame(results, columns=[column[0] for column in cursor.description])    
    
    # Close the database connection
    conn.close()

In [40]:

df_manhattan_zone = pd.read_csv('../data_manipulation/taxi_zones.csv')
df_manhattan_zone.head(50)


,Unnamed: 0,OBJECTID,Shape_Leng,the_geom,Shape_Area,zone,LocationID,borough,Unnamed: 7,Unnamed: 8,...,Unnamed: 6947,Unnamed: 6948,Unnamed: 6949,Unnamed: 6950,Unnamed: 6951,Unnamed: 6952,Unnamed: 6953,Unnamed: 6954,Unnamed: 6955,Unnamed: 6956
0,4,4,0.043567,MULTIPOLYGON (((-73.97177410965318 40.72582128...,0.000112,Alphabet City,4,Manhattan,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10,24,0.047000,MULTIPOLYGON (((-73.95953658899997 40.79871852...,0.000061,Bloomingdale,24,Manhattan,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,13,12,0.036661,MULTIPOLYGON (((-74.01565756599994 40.70483308...,0.000042,Battery Park,12,Manhattan,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,14,13,0.050281,MULTIPOLYGON (((-74.01244109299991 40.71905767...,0.000149,Battery Park City,13,Manhattan,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,43,41,0.052793,MULTIPOLYGON (((-73.94773985499985 40.80959972...,0.000143,Central Harlem,41,Manhattan,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,44,45,0.045907,MULTIPOLYGON (((-73.99750445299988 40.71406913...,0.000091,Chinatown,45,Manhattan,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,47,42,0.092709,MULTIPOLYGON (((-73.93436121591056 40.83620061...,0.000264,Central Harlem North,42,Manhattan,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,48,43,0.099739,MULTIPOLYGON (((-73.97255352299985 40.76489773...,0.000380,Central Park,43,Manhattan,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,52,48,0.043747,MULTIPOLYGON (((-73.99117738199989 40.75532984...,0.000094,Clinton East,48,Manhattan,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,54,50,0.055748,MULTIPOLYGON (((-73.99667705784736 40.77351005...,0.000173,Clinton West,50,Manhattan,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Manipulate Data

## Split into Hour and Day of the week

In [41]:
df_venue_merged['merged_time'] = pd.to_datetime(df_venue_merged['merged_time'])

# Add 'day_of_week' column (Monday as 0)
df_venue_merged['day_of_week'] = df_venue_merged['merged_time'].dt.dayofweek

# Add 'hour_integer' column
df_venue_merged['hour_integer'] = df_venue_merged['merged_time'].dt.hour

In [42]:
print(df_venue_merged.head(10))

   venue_id                   venue_name  busyness          venue_type  \
0  47508476  Han Dynasty Upper West Side         0  CHINESE_RESTAURANT   
1  47508476  Han Dynasty Upper West Side         0  CHINESE_RESTAURANT   
2  47508476  Han Dynasty Upper West Side         0  CHINESE_RESTAURANT   
3  47508476  Han Dynasty Upper West Side         0  CHINESE_RESTAURANT   
4  47508476  Han Dynasty Upper West Side         0  CHINESE_RESTAURANT   
5  47508476  Han Dynasty Upper West Side        20  CHINESE_RESTAURANT   
6  47508476  Han Dynasty Upper West Side        25  CHINESE_RESTAURANT   
7  47508476  Han Dynasty Upper West Side        30  CHINESE_RESTAURANT   
8  47508476  Han Dynasty Upper West Side        30  CHINESE_RESTAURANT   
9  47508476  Han Dynasty Upper West Side        30  CHINESE_RESTAURANT   

                                    venue_address         merged_time  \
0  215 W 85th St New York, NY 10024 United States 2023-06-19 06:00:00   
1  215 W 85th St New York, NY 10024 Uni

## Grouping Venue Types

In [43]:
venue_mapping = {
    'PARK': 'Park',
    'TOURIST_DESTINATION': 'Tourist Destination',
    'MUSEUM': 'Cultural Heritage',
    'HISTORICAL':'Cultural Heritage',
    'SCENIC_POINT': 'Scenic Landmarks',
    'BRIDGE': 'Scenic Landmarks',
    'NATURE_RESERVE': 'Nature Attractions',
    'ZOO': 'Nature Attractions',
    'BOTANICAL_GARDEN': 'Nature Attractions',
    'ARTS': 'Art',
    'DESSERT':'Art',
    'CHURCH': 'Religious',
    'SYNAGOGUE':'Religious',
    'VISITOR_CENTER': 'Tourist Destination',
    'LIBRARY':'Library',
    'SHOPPING_CENTER': 'Shopping Center',
    'APPAREL':'Fashion Convenience',
    'OTHER': 'Tourist Destination',
    'SHOPPING': 'Fashion Convenience',
    'CONVENIENCE_STORE':'Neighborhood Market',
    'SUPERMARKET': 'Neighborhood Market',
    'GROCERY':'Neighborhood Market',
    'MARKET':'Neighborhood Market',
    'GIFTS': 'Gifts & Souvenirs',
    'SOUVENIR_SHOP':'Gifts & Souvenirs',
    'RESTAURANT': 'Food',
    "CHINESE_RESTAURANT": 'Food',
    "AMERICAN_RESTAURANT": 'Food',
    "BREAKFAST_RESTAURANT": 'Food',
    "ITALIAN_RESTAURANT": 'Food',
    "FRENCH_RESTAURANT": "Food",
    "ITALIAN_RESTAURANT":"Food",
    "MEDITERANEAN_RESTAURANT":"Food",
    "THAI_RESTAURANT":"Food",
    "JAPANESE_RESTAURANT":"Food",
    "SEAFOOD_RESTAURANT":"Food",
    "VEGETERIAN_RESTAURANT":"Food",
    'MEXICAN_RESTAURANT':"Food",
    'PIZZA_RESTAURANT':"Food",
    'ASIAN_RESTAURANT':"Food",
    'INDIAN_RESTAURANT':"Food",
    'BURGER_RESTAURANT':"Food",
    'DESSERT':"Food",
    'SUSHI_RESTAURANT':"Food",
    'RAMEN_RESTAURANT':"Food"
}

df_venue_static['venue_mod_type'] = df_venue_static['venue_type'].replace(venue_mapping)

In [44]:
venue_to_zone_dict = {}

# Iterate through each row in the DataFrame
for index, row in df_venue_static.iterrows():
    venue_type = row['venue_mod_type']
    zone = row['zone_id']
    
    # If the venue_type is already in the dictionary, append the zone to its list
    if venue_type in venue_to_zone_dict:
        venue_to_zone_dict[venue_type].append(zone)
    # If the venue_type is not in the dictionary, create a new entry with the zone as a list
    else:
        venue_to_zone_dict[venue_type] = [zone]

print(venue_to_zone_dict)

{'Food': [239, 142, 230, 142, 161, 239, 238, 162, 239, 239, 142, 239, 163, 263, 161, 48, 239, 237, 239, 239, 50, 230, 142, 239, 79, 239, 164, 239, 238, 238, 239, 239, 230, 238, 161, 239, 239, 239, 239, 142, 48, 236, 236, 239, 239, 238, 238, 142, 100, 163, 239, 100, 239, 239, 231, 239, 142, 238, 142, 230, 239, 238, 238, 243, 239, 163, 230, 158, 239, 231, 48, 164, 142, 239, 142, 249, 230, 107, 142, 239, 236, 239, 239, 239, 239, 239, 249, 142, 79, 239, 239, 79, 236, 239, 238, 142, 162, 237, 142, 239, 239, 142, 239, 236, 239, 238, 263, 239, 148, 239, 48, 142, 162, 239, 68, 141, 142, 239, 48, 249, 239, 239, 142, 236, 48, 142, 161, 142, 74, 142, 142, 239, 142, 48, 48, 230, 125, 142, 234, 238, 114, 48, 239, 230, 239, 142, 239, 238, 141], 'Tourist Destination': [43, 170, 186, 161, 43, 166, 144, 239, 43, 230, 164, 43, 230, 43, 246, 162, 230, 50, 43, 43, 43, 161, 236, 170, 246], 'Cultural Heritage': [43, 24, 163, 163, 236, 239, 103, 162, 239, 43, 239, 50, 170, 161, 236, 151, 43, 239, 158, 236, 4

In [45]:
df_venue_static.head(50)

,original_ven_id,hash_ven_id,latitude,longitude,zone_id,name,address,venue_type,rating,price,type_mod,zone_group,image,description,location_id,venue_mod_type
0,ven_30356e77514c7172685648526b6f77595249473851...,47508476,40.7875,-73.9764,239,Han Dynasty Upper West Side,"215 W 85th St New York, NY 10024 United States",CHINESE_RESTAURANT,4.4,2.0,Chinese Restaurant,Upper West Side,https://images.getbento.com/accounts/845ffc258...,"Trendy area with cafes, restaurants and fashio...",143368,Food
1,ven_30365f61684a68514c7730526b6f775a6c6b7a7072...,88409575,40.7770,-73.9785,142,Friedman’s,"50 W 72nd St New York, NY 10023 United States",AMERICAN_RESTAURANT,4.4,2.0,American Restaurant,Upper West Side,https://lh3.googleusercontent.com/p/AF1QipM8N3...,Friedman’s is a local restaurant and bar locat...,0,Food
2,ven_303749644b67576e694b55526b6f7759685f4a4457...,1030056,40.7758,-73.9752,43,Strawberry Fields,"Q2GG+83 New York, NY 10019 United States",TOURIST_DESTINATION,4.6,-1.0,Tourist Destination,Upper West Side,https://s3.amazonaws.com/assets.centralparknyc...,Strawberry Fields is a music venue located in ...,0,Tourist Destination
3,ven_3038654c546f3936784873526b6f775a3148646771...,23854076,40.7580,-73.9853,230,Time Square,"199 W 45th St New York, NY 10036 United States",RESTAURANT,4.7,-1.0,Restaurant,Midtown Manhattan,https://upload.wikimedia.org/wikipedia/commons...,Time Square is a world-famous entertainment di...,0,Food
4,ven_3039793356676f745a3570526b6f775974496c6d73...,20848781,40.7776,-73.9784,142,TAP NYC | 100% Gluten-Free Sandwiches & Açaí B...,"267 Columbus Ave New York, NY 10023 United States",RESTAURANT,4.6,2.0,Restaurant,Upper West Side,https://lh3.googleusercontent.com/p/AF1QipNByb...,TAP NYC is a 100% gluten-free sandwich and aça...,0,Food
5,ven_304330524e695f70503476526b6f775a6c6e5a7133...,43331760,40.7630,-73.9767,161,The Consulate,"44 W 56th St New York, NY 10019 United States",FRENCH_RESTAURANT,4.8,-1.0,French Restaurant,Midtown Manhattan,https://s3-media0.fl.yelpcdn.com/bphoto/zvwIqh...,"The Consulate is a large, modern venue located...",0,Food
6,ven_3046564f552d3633755343526b6f77595649495368...,77654852,40.7857,-73.9726,239,The Viand,"517 Columbus Ave New York, NY 10024 United States",BREAKFAST_RESTAURANT,4.3,2.0,Breakfast Restaurant,Upper West Side,https://media-cdn.tripadvisor.com/media/photo-...,"The Viand is a restaurant located in New York,...",0,Food
7,ven_3048674a4764455f76744f526b6f77594a6f5f6176...,82820658,40.7911,-73.9740,238,Carmine's Italian Restaurant - Upper West Side,"2450 Broadway New York, NY 10024 United States",ITALIAN_RESTAURANT,4.4,2.0,Italian Restaurant,Upper West Side,https://media-cdn.tripadvisor.com/media/photo-...,Carmine's Italian Restaurant - Upper West Side...,0,Food
8,ven_304961777454515336474c526b6f775a4e51737357...,55261692,40.7512,-73.9749,162,The Capital Grille,"155 E 42nd St New York, NY 10017 United States",RESTAURANT,4.6,4.0,Restaurant,Midtown Manhattan,https://newyorklifestylesmagazine.com/images/c...,The Capital Grille is an upscale steakhouse wi...,0,Food
9,ven_304a6c77747a366c755455526b6f77594a70705759...,57103262,40.7739,-73.9710,43,Bethesda Terrace,"New York, NY 10024 United States",HISTORICAL,4.7,-1.0,Cultural Heritage,Upper West Side,https://upload.wikimedia.org/wikipedia/commons...,Bethesda Terrace is an iconic New York City la...,10067208,Cultural Heritage


# Clearing Duplicates

In [46]:
# look for duplicates
print('Number of duplicate (excluding first) rows in the table is: ', df_venue_static.duplicated().sum())

# use "keep=False" to mark all duplicates as true, including the original rows that were duplicated
print('Number of duplicate rows (including first) in the table is:', df_venue_static[df_venue_static.duplicated(keep=False)].shape[0])

Number of duplicate (excluding first) rows in the table is:  0
Number of duplicate rows (including first) in the table is: 0


In [47]:
# look for duplicates
print('Number of duplicate (excluding first) rows in the table is: ', df_venue_timings.duplicated().sum())

# use "keep=False" to mark all duplicates as true, including the original rows that were duplicated
print('Number of duplicate rows (including first) in the table is:', df_venue_timings[df_venue_timings.duplicated(subset=['venue_id', 'day', 'opening_time', 'closing_time'], keep='first')].shape[0])

Number of duplicate (excluding first) rows in the table is:  0
Number of duplicate rows (including first) in the table is: 777


In [48]:
print('Number of duplicate (excluding first) rows in the table is: ', df_venue_timings.drop_duplicates(subset=['venue_id', 'day', 'opening_time', 'closing_time'], inplace=True))
df_venue_timings

Number of duplicate (excluding first) rows in the table is:  None


,id,venue_id,day,opening_time,closing_time
0,1,ven_30356e77514c7172685648526b6f77595249473851...,0,11,22
1,2,ven_30356e77514c7172685648526b6f77595249473851...,1,11,22
2,3,ven_30356e77514c7172685648526b6f77595249473851...,2,11,22
3,4,ven_30356e77514c7172685648526b6f77595249473851...,3,11,22
4,5,ven_30356e77514c7172685648526b6f77595249473851...,4,11,22
...,...,...,...,...,...
3250,3629,ven_77446a687a695a67306746526b6f77595a6c644878...,2,8,0
3251,3630,ven_77446a687a695a67306746526b6f77595a6c644878...,3,8,0
3252,3631,ven_77446a687a695a67306746526b6f77595a6c644878...,4,8,0
3253,3632,ven_77446a687a695a67306746526b6f77595a6c644878...,5,8,1


# Grouping Zones

In [49]:
venue_zone_grouping = {
    'Upper Manhattan': [128, 127, 243, 120, 244, 116, 42, 152, 41, 74, 75],
    'Upper West Side': [166, 24, 151, 43, 238, 239, 143, 142],
    'Upper East Side': [236,263, 262, 237, 141, 140 ],
    'Chelsea/Greenwhich market':[246, 68, 186, 90, 100, 234, 158, 249, 113, 249],
    'Lower Manhattan': [107, 224, 114, 211, 144, 148, 232, 231, 45, 13, 261, 209, 87, 88, 12 ],
    'Midtown Manhattan': [50, 48, 230, 163, 161, 162, 229, 233, 164, 170, 137, 224, 107, 234]
}


In [50]:
# venue_zone_grouping dictionary
venue_zone_grouping = {
    'Upper Manhattan': [128, 127, 243, 120, 244, 116, 42, 152, 41, 74, 75],
    'Upper West Side': [166, 24, 151, 43, 238, 239, 143, 142],
    'Upper East Side': [236, 263, 262, 237, 141, 140],
    'Chelsea/Greenwhich market': [246, 68, 186, 90, 100, 234, 158, 249, 113, 249],
    'Lower Manhattan': [107, 224, 114, 211, 144, 148, 232, 231, 45, 13, 261, 209, 87, 88, 12],
    'Midtown Manhattan': [50, 48, 230, 163, 161, 162, 229, 233, 164, 170, 137, 224, 107, 234],
}

# Function to map zone numbers to zone groups
def map_zone_group(zone_number):
    for zone_group, zone_numbers in venue_zone_grouping.items():
        if zone_number in zone_numbers:
            return zone_group
    return 'Other'  # If zone number not found in the dictionary, assign 'Other'

# Create the 'zone_group' column based on the mapping
df_venue_static['zone_group'] = df_venue_static['zone_id'].apply(map_zone_group)

print(df_venue_static)


                                       original_ven_id  hash_ven_id  latitude  \
0    ven_30356e77514c7172685648526b6f77595249473851...     47508476   40.7875   
1    ven_30365f61684a68514c7730526b6f775a6c6b7a7072...     88409575   40.7770   
2    ven_303749644b67576e694b55526b6f7759685f4a4457...      1030056   40.7758   
3    ven_3038654c546f3936784873526b6f775a3148646771...     23854076   40.7580   
4    ven_3039793356676f745a3570526b6f775974496c6d73...     20848781   40.7776   
..                                                 ...          ...       ...   
351  ven_7768585a615270346e454b526b6f7766426c2d4f31...     40246733   40.7797   
352  ven_77686b76675f51577a7053526b6f775a356b526546...     76894533   40.7694   
353  ven_776c426b3172735a6f5455526b6f77595a5f4c7350...     96046596   40.7682   
354  ven_7774565736353150745f5f526b6f775a3949484637...     12087255   40.7244   
355  ven_7778775545424e486d7075526b6f7759465048596d...     31643001   40.7666   

     longitude  zone_id    

In [51]:
df_venue_static.to_csv('zone_Grouping.csv', index=False)

# Extracting Attraction Types 

In [52]:
unique_type_values = df_venue_static['venue_mod_type'].unique()
unique_type_values

array(['Food', 'Tourist Destination', 'Cultural Heritage',
       'Fashion Convenience', 'BAR', 'Park', 'Neighborhood Market',
       'Shopping Center', 'Scenic Landmarks', 'Nature Attractions',
       'Gifts & Souvenirs', 'BEER', 'FISHING', 'Art', 'Religious',
       'Library'], dtype=object)

In [53]:
specific_venue_types = ['Nature Attractions', 'Shopping Center', 'Tourist Destination', 'Cultural Heritage', 'Neighborhood Market', 'Fashion Convenience', 'Library', 'Scenic Landmarks', 'Art', 'Religious', 'Park', 'Gifts & Souvenirs', 'Food']

# Filter the DataFrame to only include rows with the specific venue types
df_venue_static_att = df_venue_static[df_venue_static['venue_mod_type'].isin(specific_venue_types)]

# Now 'filtered_df' contains only rows where the "Attraction_Type" is in the specified list
print(df_venue_static_att)

                                       original_ven_id  hash_ven_id  latitude  \
0    ven_30356e77514c7172685648526b6f77595249473851...     47508476   40.7875   
1    ven_30365f61684a68514c7730526b6f775a6c6b7a7072...     88409575   40.7770   
2    ven_303749644b67576e694b55526b6f7759685f4a4457...      1030056   40.7758   
3    ven_3038654c546f3936784873526b6f775a3148646771...     23854076   40.7580   
4    ven_3039793356676f745a3570526b6f775974496c6d73...     20848781   40.7776   
..                                                 ...          ...       ...   
351  ven_7768585a615270346e454b526b6f7766426c2d4f31...     40246733   40.7797   
352  ven_77686b76675f51577a7053526b6f775a356b526546...     76894533   40.7694   
353  ven_776c426b3172735a6f5455526b6f77595a5f4c7350...     96046596   40.7682   
354  ven_7774565736353150745f5f526b6f775a3949484637...     12087255   40.7244   
355  ven_7778775545424e486d7075526b6f7759465048596d...     31643001   40.7666   

     longitude  zone_id    

# Actual Flow

### Input

In [54]:
user_zone_input = ['Upper Manhattan','Midtown Manhattan']

In [55]:
user_input_attractions = ['Fashion Convenience', 'Park', 'Tourist Destination']

In [56]:
unique_type_values_att = df_venue_static_att['venue_mod_type'].unique()
unique_type_values_att

array(['Food', 'Tourist Destination', 'Cultural Heritage',
       'Fashion Convenience', 'Park', 'Neighborhood Market',
       'Shopping Center', 'Scenic Landmarks', 'Nature Attractions',
       'Gifts & Souvenirs', 'Art', 'Religious', 'Library'], dtype=object)

In [57]:
if len(user_input_attractions) < 4:
    x = 4 - len(user_input_attractions)
    updated_list = [num for num in unique_type_values_att if num not in user_input_attractions]
    
    similarities = []
    user_input_tag_embedding = nlp(user_input_attractions[0]).vector

    for tag in updated_list:
        tag_embedding = nlp(tag).vector
        similarity = user_input_tag_embedding.dot(tag_embedding) / (np.linalg.norm(user_input_tag_embedding) * np.linalg.norm(tag_embedding))
        similarities.append(similarity)

    sorted_indices = np.argsort(similarities)[::-1]  # Descending order
    most_similar_tags = [updated_list[i] for i in sorted_indices]
    slice_most_similar_tags = most_similar_tags[0:x]
    user_input_attractions = user_input_attractions + slice_most_similar_tags

In [58]:
user_input_attractions

['Fashion Convenience', 'Park', 'Tourist Destination', 'Nature Attractions']

In [59]:
# df_food.head()

In [60]:
import pandas as pd
from datetime import datetime, timedelta

# Create the priority table
priority_table = pd.DataFrame({
    'Attraction': ['Parks', 'Tourist Destination', 'Cultural Heritage', 'Scenic Landmarks', 
                   'Nature Attractions', 'Religious', 'Art', 'Library', 'Shopping Center', 
                   'Fashion Convenience', 'Neighborhood Market', 'Gifts & Souvenirs', 'Food'],
    'Opening_Time': ['9:00 AM', '9:00 AM', '11:00 AM', '9:00 AM', '10:00 AM', '11:00 AM', '9:00 AM', 
                     '9:00 AM', '10:00 AM', '10:00 AM', '10:00 AM', '10:00 AM', '8:00 AM'],
    'Closing_Time': ['6:00 PM', '6:00 PM', '5:00 PM', '1:00 AM', '4:00 PM', '5:00 PM', '5:00 PM', 
                     '7:00 PM', '6:00 PM', '6:00 PM', '6:00 PM', '6:00 PM', '10:00 PM']
})

# Convert the opening and closing times to datetime format
priority_table['Opening_Time'] = pd.to_datetime(priority_table['Opening_Time']).dt.time
priority_table['Closing_Time'] = pd.to_datetime(priority_table['Closing_Time']).dt.time

# Define the user_input_attractions (example)
# user_input_attractions = ['Parks', 'Art']

# Filter the priority table for the user's chosen attractions
filtered_priority = priority_table[priority_table['Attraction'].isin(user_input_attractions)].sort_values('Opening_Time')

# Add the Food attraction for breaks
food_attraction = priority_table[priority_table['Attraction'] == 'Food'].iloc[0]

# Create the suggested itinerary
itinerary = {}
current_time = datetime.combine(datetime.today(), datetime.strptime('09:00 AM', '%I:%M %p').time())

for index, row in filtered_priority.iterrows():
    attraction = row['Attraction']
    opening_time = row['Opening_Time']
    closing_time = row['Closing_Time']

    if opening_time <= current_time.time() <= closing_time:
        # Let's assume the user spends 2 hours at each attraction
        end_visit_time = (current_time + timedelta(hours=2)).time()

        if end_visit_time <= closing_time:
            itinerary[attraction] = f"{current_time.strftime('%I:%M %p')} - {datetime.strptime(str(end_visit_time), '%H:%M:%S').strftime('%I:%M %p')}"
            current_time = datetime.combine(datetime.today(), end_visit_time)

        # Add food break after each attraction visit, but only if it's before food closing time
        food_end_time = (current_time + timedelta(hours=1)).time()
        if food_end_time <= food_attraction['Closing_Time']:
            itinerary['Food Break'] = f"{current_time.strftime('%I:%M %p')} - {datetime.strptime(str(food_end_time), '%H:%M:%S').strftime('%I:%M %p')}"
            current_time = datetime.combine(datetime.today(), food_end_time)

# Print the suggested itinerary
print("Suggested Itinerary:")
for attraction, timing in itinerary.items():
    print(f"{attraction}: {timing}")

Suggested Itinerary:
Tourist Destination: 09:00 AM - 11:00 AM
Food Break: 05:00 PM - 06:00 PM
Nature Attractions: 12:00 PM - 02:00 PM
Fashion Convenience: 03:00 PM - 05:00 PM


In [61]:
zone_group = []
for group in user_zone_input:
    for zone in venue_zone_grouping[group]:
        zone_group.append(zone)
zone_group

[128,
 127,
 243,
 120,
 244,
 116,
 42,
 152,
 41,
 74,
 75,
 50,
 48,
 230,
 163,
 161,
 162,
 229,
 233,
 164,
 170,
 137,
 224,
 107,
 234]

In [62]:
user_venue_per_type_dict = {}
for venue_type in itinerary_timing:
    matched_zones = df_venue_static_att[df_venue_static_att['venue_mod_type'] == venue_type]['zone_id']
    matching_zones = matched_zones[matched_zones.isin(zone_group)]
    result_df = df_venue_static_att[df_venue_static_att['zone_id'].isin(matching_zones)]['original_ven_id']
    user_venue_per_type_dict[venue_type] = list(result_df)

# Print the resulting dictionary
print(user_venue_per_type_dict)

NameError: name 'itinerary_timing' is not defined

In [ ]:
# Need to check for empty Type

In [ ]:
zone_type_dict = {}
for venue_type in itinerary_timing:
    matched_zones = df_venue_static_att[df_venue_static_att['venue_mod_type'] == venue_type]['zone_id'].unique()
    zone_type_dict[venue_type] = list(matched_zones)

# Print the resulting dictionary
print(zone_type_dict)

{'Tourist Destination': [43, 170, 186, 161, 166, 144, 239, 230, 164, 246, 162, 50, 236], 'Nature Attractions': [43], 'Shopping Center': [74, 246, 234, 163, 162, 113, 142, 143], 'Fashion Convenience': [263, 236, 230, 237, 144, 141, 163, 114, 142, 164, 90, 48, 161, 232, 239]}


In [ ]:
types_with_zero_zones = []

# Iterate through the venue_type_dict
for venue_type, zones in zone_type_dict.items():
    if len(zones) == 0:
        types_with_zero_zones.append(venue_type)

print("Venue types with 0 zones:", types_with_zero_zones)

Venue types with 0 zones: []


In [ ]:
def find_distance_between_zones(zone1_polygon, zone2_polygon):
    # Find the nearest points between the two polygons
    nearest_points_result = nearest_points(wkt.loads(zone1_polygon), wkt.loads(zone2_polygon))

    # Calculate the distance between the nearest points
    distance = nearest_points_result[0].distance(nearest_points_result[1])
    return distance

In [ ]:
filled_type_with_zero_zone = {}

for ven_type in types_with_zero_zones:
    venue_to_zone_dict_copy = list(set(venue_to_zone_dict[i]))
    if len(venue_to_zone_dict_copy) == 0:
        #get all venue id of each zone and push it to user_venue_per_type_dict of that type
        print('okay')
    else:
        zone_between_dist = []
        for user_zone in zone_group:
            for venue_zone in venue_to_zone_dict_copy:
                zone1_polygon = df_manhattan_zone[df_manhattan_zone['LocationID'] == user_zone]['the_geom'].iloc[0]
                zone2_polygon = df_manhattan_zone[df_manhattan_zone['LocationID'] == venue_zone]['the_geom'].iloc[0]
                distance = find_distance_between_zones(zone1_polygon, zone2_polygon)
                zone_between_dist.append((user_zone, venue_zone, distance)) 
        sorted_zone_between_dist = sorted(zone_between_dist, key=lambda x: x[2])[:3]
        new_zone = [df_venue_static_att[
            (df_venue_static_att['zone_id'] == item[1]) &
            (df_venue_static_att['venue_mod_type'] == ven_type)
        ]['original_ven_id'].tolist() for item in sorted_zone_between_dist]
        
        print(new_zone)
        filled_type_with_zero_zone[ven_type] = list(set(item for sublist in new_zone for item in sublist))

filled_type_with_zero_zone

{}

In [ ]:
for key in user_venue_per_type_dict.keys():
    # Check if the value of the current key is an empty array
    if len(user_venue_per_type_dict[key]) == 0:
        # Check if the key exists in dictionary B
        if key in filled_type_with_zero_zone:
            # Replace the value in A with the value from B
            user_venue_per_type_dict[key] = filled_type_with_zero_zone[key]
print(user_venue_per_type_dict)

{'Tourist Destination': ['ven_3038654c546f3936784873526b6f775a314864677151534a496843', 'ven_304330524e695f70503476526b6f775a6c6e5a713332574a496843', 'ven_304961777454515336474c526b6f775a4e5173735733524a496843', 'ven_30782d4b5379586f66706d526b6f775a745f45477777774a496843', 'ven_342d4f33343573744c7a2d526b6f775a42774547545f774a496843', 'ven_344d7478767a4b74377548526b6f7759566c78494a516d4a496843', 'ven_346c6843676c5559377662526b6f775a354f49314937644a496843', 'ven_3855585f5861474e706f34526b6f7759393079333631484a496843', 'ven_385a743953644e70587637526b6f7759395f534f5334334a496843', 'ven_386b4c712d792d70556e33526b6f77596c5f66384652324a496843', 'ven_4148494f684649484f4477526b6f775a4e68634d4255554a496843', 'ven_41633250634c684c2d2d78526b6f775956567774752d6b4a496843', 'ven_4168437230416a45783158526b6f7759526c2d6f5339524a496843', 'ven_417239716752736e6c447a526b6f7759745078672d574d4a496843', 'ven_453338504841477742644e526b6f776135556f514566504a496843', 'ven_454e4e686e4a7046453659526b6f775a6c367352

In [ ]:
time_index = 0
filtered_venues = {}

# Loop through each venue type and check for the specified condition
for venue_type, venue_ids in user_venue_per_type_dict.items():
    valid_venues = []
    for venue_id in venue_ids:
        venue_info = df_venue_timings.loc[df_venue_timings['venue_id'] == venue_id]
        filtered_venue_info = venue_info.loc[(venue_info['day'] == 6) & (venue_info['opening_time'] <= opening_time[time_index]) 
#                                              & (venue_info['closing_time'] >= closing_time[time_index])
                                            ]
        if not filtered_venue_info.empty:
            valid_venues.append(venue_id)
    if valid_venues:
        filtered_venues[venue_type] = valid_venues
        time_index+=1
print(filtered_venues)

In [ ]:
manipulated_venues = {}
introvert_mode = True  # Set to False if introvert mode is off

# Loop through each venue type and check for the specified condition
for venue_type, venue_ids in user_venue_per_type_dict.items():
#     if venue_type == 'Food':
#         # If venue_type is 'Food', get a food venue and add it to manipulated_venues
#         current_time = pd.Timestamp.now()  # or whatever the current time is in your specific implementation
#         food_venue = get_open_food_venue(df_food, current_time, introvert_mode)
#         manipulated_venues[venue_type] = [(food_venue['name'], food_venue['rating'], food_venue['busyness'], 0)]  # 0 for composite_score as it's not calculated for food venues
#     else:
#         # If venue_type is not 'Food', proceed as before
    valid_venues = []
    for venue_id in venue_ids:
        venue_hash_id = df_venue_static.loc[df_venue_static['original_ven_id'] == venue_id]['hash_ven_id']
        venue_rating = df_venue_static.loc[df_venue_static['original_ven_id'] == venue_id]['rating'].item()
        venue_hash_id = int(venue_hash_id)
        df_venue_merged['venue_id'] = df_venue_merged['venue_id'].astype(int)
        specific_venue_df = df_venue_merged.loc[df_venue_merged['venue_id'] == venue_hash_id]
        average_busyness = specific_venue_df['busyness'].mean() 
        
        weight_rating = 0.4 if introvert_mode else 0.6
        weight_busyness = 0.6 if introvert_mode else 0.4
       # Invert the busyness score if introvert mode is on
        average_busyness = 1 - average_busyness if introvert_mode else average_busyness

        composite_score = (weight_rating * venue_rating) + (weight_busyness * average_busyness)
        
        
        valid_venues.append((venue_id, venue_rating, average_busyness, composite_score))
    if valid_venues:
        manipulated_venues[venue_type] = valid_venues

manipulated_venues

{'Tourist Destination': [('ven_30782d4b5379586f66706d526b6f775a745f45477777774a496843',
   5.0,
   -34.67424242424242,
   -18.80454545454545),
  ('ven_344d7478767a4b74377548526b6f7759566c78494a516d4a496843',
   4.4,
   -20.954545454545453,
   -10.812727272727273),
  ('ven_346c6843676c5559377662526b6f775a354f49314937644a496843',
   4.5,
   -40.43181818181818,
   -22.459090909090907),
  ('ven_386b4c712d792d70556e33526b6f77596c5f66384652324a496843',
   4.5,
   -17.022727272727273,
   -8.413636363636362),
  ('ven_41633250634c684c2d2d78526b6f775956567774752d6b4a496843',
   4.6,
   -14.863636363636363,
   -7.078181818181818),
  ('ven_453338504841477742644e526b6f776135556f514566504a496843',
   4.3,
   -16.696969696969695,
   -8.298181818181817),
  ('ven_454e4e686e4a7046453659526b6f775a6c3673525158614a496843',
   4.7,
   -39.18181818181818,
   -21.62909090909091),
  ('ven_4565635872326c5643414d526b6f775a70364b4e5452304a496843',
   4.3,
   -13.878787878787879,
   -6.6072727272727265),
  ('ven_4

In [ ]:

user_pref_rating = input("On a scale of 1 to 5, how important are venue ratings to you?")
user_pref_busyness = input("On a scale of 1 to 5, how important is venue busyness to you?")


weight_rating = float(user_pref_rating) / 5
weight_busyness = float(user_pref_busyness) / 5


In [ ]:

# user_feedback = input("On a scale of 1 to 5, how would you rate the recommended itinerary?")

In [ ]:
top_3_venues = {}

# Loop through each venue type and its venues
for venue_type, venue_data in manipulated_venues.items():
    # Sort the venues based on the composite score (fourth element in the tuple, index 3)
    if len(venue_data) > 3:
        sorted_venues = sorted(venue_data, key=lambda x: x[3], reverse=True)
    
        # Keep only the top 3 venues for each venue type
        top_3_venues[venue_type] = sorted_venues[:3]
    else:
        top_3_venues[venue_type] = venue_data

# Display the top 3 venues for each venue type
for venue_type, top_venues in top_3_venues.items():
    print(f"Venue Type: {venue_type}")
    for rank, (venue_id, rating, busyness, score) in enumerate(top_venues, start=1):
        print(f"Rank {rank}: Venue ID: {venue_id}, Rating: {rating}, Busyness: {busyness}, Score: {score}")
    print()

Venue Type: Tourist Destination
Rank 1: Venue ID: ven_4d6a306b416131617a736f526b6f7759355f7a6d3155394a496843, Rating: 4.7, Busyness: 43.38636363636363, Score: 20.174545454545456
Rank 2: Venue ID: ven_346c6843676c5559377662526b6f775a354f49314937644a496843, Rating: 4.5, Busyness: 41.43181818181818, Score: 19.272727272727273
Rank 3: Venue ID: ven_454e4e686e4a7046453659526b6f775a6c3673525158614a496843, Rating: 4.7, Busyness: 40.18181818181818, Score: 18.892727272727274

Venue Type: Fashion Convenience
Rank 1: Venue ID: ven_34416b6a74734635334274526b6f775964314e46695a734a496843, Rating: 4.4, Busyness: 52.46212121212121, Score: 23.624848484848485
Rank 2: Venue ID: ven_4d6a306b416131617a736f526b6f7759355f7a6d3155394a496843, Rating: 4.7, Busyness: 43.38636363636363, Score: 20.174545454545456
Rank 3: Venue ID: ven_346c6843676c5559377662526b6f775a354f49314937644a496843, Rating: 4.5, Busyness: 41.43181818181818, Score: 19.272727272727273

Venue Type: Food
Rank 1: Venue ID: ven_34416b6a74734635334